In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [2]:
A=np.array([[3,2000],
            [2,3000],
            [4,5000],
            [5,8000],
            [1,2000]],dtype='float')

#数据归一化
mean=np.mean(A,axis=0)
norm=A-mean
# 数据缩放
scope=np.max(norm,axis=0) - np.min(norm,axis=0)
norm=norm/scope
norm

array([[ 0.        , -0.33333333],
       [-0.25      , -0.16666667],
       [ 0.25      ,  0.16666667],
       [ 0.5       ,  0.66666667],
       [-0.5       , -0.33333333]])

# PCA算法模拟:利用奇异值分解

In [3]:
#计算协方差矩阵
C=np.dot(norm.T,norm)
C

array([[ 0.625     ,  0.58333333],
       [ 0.58333333,  0.72222222]])

In [4]:
#对待下列方差矩阵进行奇异值分解  得到特征向量  U
U,S,V=np.linalg.svd(C)
U

array([[-0.67710949, -0.73588229],
       [-0.73588229,  0.67710949]])

In [5]:
#因为要将二维转一维  所以只取特征矩阵的第一列来构造主成分矩阵
U_reduce=U[:,0].reshape(2,1)
U_reduce

array([[-0.67710949],
       [-0.73588229]])

In [6]:
#利用主成分特征矩阵来进行降维处理  得到的结果是一个一维矩阵
Z = np.dot(norm,U_reduce)
Z

array([[ 0.2452941 ],
       [ 0.29192442],
       [-0.29192442],
       [-0.82914294],
       [ 0.58384884]])

In [7]:
Xapprox=np.dot(Z,U_reduce.T)
Xapprox

array([[-0.16609096, -0.18050758],
       [-0.19766479, -0.21482201],
       [ 0.19766479,  0.21482201],
       [ 0.56142055,  0.6101516 ],
       [-0.39532959, -0.42964402]])

In [8]:
np.multiply(Xapprox,scope)+mean   #乘 缩放比   +  均值

array([[  2.33563616e+00,   2.91695452e+03],
       [  2.20934082e+00,   2.71106794e+03],
       [  3.79065918e+00,   5.28893206e+03],
       [  5.24568220e+00,   7.66090960e+03],
       [  1.41868164e+00,   1.42213588e+03]])

# 使用  sklearn  实现

In [9]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

n=1
scaler=MinMaxScaler()
arr=scaler.fit_transform(A)

pca=PCA( n_components=n)
R2=pca.fit_transform(arr)
R2

array([[-0.2452941 ],
       [-0.29192442],
       [ 0.29192442],
       [ 0.82914294],
       [-0.58384884]])

In [10]:
#输出其他结果

print('保留的成分个数:',pca.n_components_)
print('特征的方差:',pca.components_)
#  代表降维后的各成分的方差值占总方差的比例  这个比例越大，则越是重要的主要成分
print('保留的',n,'个成分,降维后的各主成分的方差值占总方差值的比例,这个比例越大，则越是重要的主要分',pca.explained_variance_ratio_)
print('代表降维后的各成分的方差值。方差值越大，则说明越是重要的主成分',pca.explained_variance_)

保留的成分个数: 1
特征的方差: [[ 0.67710949  0.73588229]]
保留的 1 个成分,降维后的各主成分的方差值占总方差值的比例,这个比例越大，则越是重要的主要分 [ 0.93449053]
代表降维后的各成分的方差值。方差值越大，则说明越是重要的主成分 [ 0.3147416]


In [11]:
def std_PCA(**argv):
    '''
    利用管道进行数据缩放后加入到PCA处理
    '''
    scaler = MinMaxScaler()
    pca=PCA(**argv)
    pipeline=Pipeline([('scaler',scaler),('pca',pca)])
    return pipeline

pca=std_PCA(n_components=1)
R3 = pca.fit_transform(A)
R3

array([[-0.2452941 ],
       [-0.29192442],
       [ 0.29192442],
       [ 0.82914294],
       [-0.58384884]])

In [12]:
pca.inverse_transform(R2)   #再恢复成近似的原值

array([[  2.33563616e+00,   2.91695452e+03],
       [  2.20934082e+00,   2.71106794e+03],
       [  3.79065918e+00,   5.28893206e+03],
       [  5.24568220e+00,   7.66090960e+03],
       [  1.41868164e+00,   1.42213588e+03]])

In [13]:
import matplotlib.font_manager as fm

plt.figure(figsize=(5,5),dpi=144)
plt.title('PCA的物理意义',fontproperties=myfont)

ymin=xmin=-1
ymax=xmax=1
plt.

# PCA算法模拟:2.计算特征值和特征向量来实现降维

In [15]:
#过程演示
meanValues=np.mean(norm,axis=0)
meanRemoved=A-meanValues
#numpy中的matrix是ndarray子类
# matrix.getA  -》 ndarray
covValues=np.cov(meanRemoved,rowvar=0)  #协方差矩阵
eigValues,eigVectors=np.linalg.eig( np.mat(covValues) )  #求特征值和特征向量
print('特征值',eigValues)
print('特征向量',eigVectors)

特征值 [  6.15384437e-01   6.50000188e+06]
特征向量 [[ -9.99999855e-01  -5.38461511e-04]
 [  5.38461511e-04  -9.99999855e-01]]


In [16]:
#pca 封装函数
def pca(dataMatrics,topNfeat=4096):
    '''
    dataMatrics  待降维矩阵
    topNfeat    要保留的特征数
    '''
    #求数据矩阵每一列的均值
    meanValues=np.mean(dataMatrics,axis=0)
    #数据矩阵每一列特征减去该列的特征均值
    meanRemoved=dataMatrics-meanValues
    #计算协方差矩阵，除数n-1是为了得到协方差的无偏估计
    #  cov(x,0)=cov(x)  除数  n-1  （n为样本个数）
    #  cov(x,1)  除数是n
    covMatrics=np.cov(meanRemoved,rowvar=0)
    #计算协方差矩阵的特征值及其对应的特征向量均保持在相应的矩阵中
    eigValues,eigVectors=np.linalg.eig(np.mat(covMatrics) )
    #sort()  对特征值矩阵排序(由小到大)
    #argsort(): 对特征值矩阵进行由小到大排序  返回对应排序后的索引
    eigValInd=np.argsort(eigValues)
    #从排好序的矩阵最后一个开始自下而上选取最大的N个特征值  返回其对应的索引
    eigValInd=eigValInd[:-(topNfeat+1):-1]
    #将特征值最大的N个特征值对应索引的特征向量提取出来，组成亚索矩阵
    redEigVects=eigVectors[:,eigValInd]
    #将去除均值后的   数据矩阵*压缩矩阵  转换到新的空间，使维度降低到N
    lowDDataMatrics=meanRemoved*redEigVects
    #利用降维后的矩阵反构出原数据矩阵
    recoverMat=(lowDDataMatrics*redEigVects.T)+meanValues
    #返回值：降维后的低维矩阵,还原的矩阵
    return lowDDataMatrics,recoverMat

In [17]:
newMat,recoverMat=pca(norm,1)
print(newMat)
print(recoverMat)
recoverMat=np.multiply(recoverMat,scope)+mean
recoverMat

[[ 0.2452941 ]
 [ 0.29192442]
 [-0.29192442]
 [-0.82914294]
 [ 0.58384884]]
[[-0.16609096 -0.18050758]
 [-0.19766479 -0.21482201]
 [ 0.19766479  0.21482201]
 [ 0.56142055  0.6101516 ]
 [-0.39532959 -0.42964402]]


matrix([[  2.33563616e+00,   2.91695452e+03],
        [  2.20934082e+00,   2.71106794e+03],
        [  3.79065918e+00,   5.28893206e+03],
        [  5.24568220e+00,   7.66090960e+03],
        [  1.41868164e+00,   1.42213588e+03]])